In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.thing = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         return self.thing

In [3]:
# net = Net()

# learning_rate = 0.1

# loss_fn = nn.MSELoss()

# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [4]:
# ideal_weights = torch.tensor([0.0027] * 10)
# for epoch in range(100):
#     optimizer.zero_grad()
#     pred = net()
#     print(pred)

#     weight = torch.zeros(10)
#     for i in range(10):
#         if pred[i].item() > 0.5:
#             weight[i] = 0.0027
#         else:
#             weight[i] = 0.00783
    
#     total_loss = ideal_weights - weight
#     print(ideal_weights)
#     print(weight)
#     print(total_loss)

#     loss = loss_fn(pred, total_loss.float())
#     print(loss)
#     loss.backward()
#     optimizer.step()

In [5]:
# class customAutograd(torch.autograd.Function):
#     @staticmethod
#     def forward(ctx, input):
#         """
#         In the forward pass we receive a Tensor containing the input and return
#         a Tensor containing the output. ctx is a context object that can be used
#         to stash information for backward computation. You can cache arbitrary
#         objects for use in the backward pass using the ctx.save_for_backward method.
#         """
#         ctx.save_for_backward(input)
#         return input / 2

#     @staticmethod
#     def backward(ctx, grad_output):
#         """
#         In the backward pass we receive a Tensor containing the gradient of the loss
#         with respect to the output, and we need to compute the gradient of the loss
#         with respect to the input.
#         """
#         input, = ctx.saved_tensors
#         return grad_output * input

In [6]:
# class testNet(nn.Module):
#     def __init__(self):
#         super(testNet, self).__init__()
#         torch.manual_seed(12)
#         self.elem_material = nn.Parameter(torch.rand(10), requires_grad=True)
    
#     def forward(self):
#         print(self.elem_material)

#         E_vector = 131 / (1 + torch.exp(-10 * (self.elem_material - torch.tensor([0.5] * 10)))) + 69
#         Stif = E_vector * 10
#         u = Stif / 2

#         return u


In [7]:
# net = testNet()

# num_epochs = 10
# learning_rate = 1
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

# ideal_u = torch.tensor([5.0] * 10)

# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     u = net()

#     loss = loss_fn(u, ideal_u)
#     loss.backward()

#     for name, param in net.named_parameters():
#         if param.requires_grad:
#             print(name, param.grad)

#     optimizer.step()

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
class testFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx, mat):
        ctx.save_for_backward(mat)

        return torch.linalg.inv(mat)
    
    def backward(ctx, grad_output):
        mat = ctx.saved_tensors[0]
        print(mat)
        matKron = torch.tensor(torch.kron(mat, mat), requires_grad=True)
        matKron.backward(torch.ones_like(matKron))
    
        return torch.mm(grad_output, matKron)

In [10]:
testFunction = testFunc.apply

In [11]:
class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.mat = nn.Parameter(torch.rand((20, 20), requires_grad=True, device=device))

    def forward(self):    
        return testFunction(self.mat)

In [12]:
net = testNet()
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

target = torch.zeros((20, 20), device=device)

for epoch in range(10):
    output = net()
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

Parameter containing:
tensor([[0.9085, 0.6041, 0.9862, 0.8138, 0.0299, 0.8591, 0.7446, 0.6788, 0.5976,
         0.2402, 0.1613, 0.1781, 0.9200, 0.4087, 0.2358, 0.3741, 0.8329, 0.7011,
         0.3213, 0.7330],
        [0.6542, 0.9598, 0.9757, 0.4503, 0.6453, 0.8637, 0.3598, 0.1752, 0.0708,
         0.4350, 0.3135, 0.0548, 0.9126, 0.0697, 0.3114, 0.6958, 0.5950, 0.6693,
         0.2519, 0.7557],
        [0.6775, 0.9440, 0.6960, 0.2229, 0.7594, 0.2043, 0.8151, 0.4815, 0.0986,
         0.7851, 0.4147, 0.8952, 0.9937, 0.0763, 0.3401, 0.4406, 0.6369, 0.6385,
         0.4158, 0.8581],
        [0.6103, 0.8722, 0.3648, 0.6447, 0.3303, 0.4596, 0.3032, 0.7894, 0.8735,
         0.7091, 0.4641, 0.2607, 0.5981, 0.6969, 0.6419, 0.9279, 0.0665, 0.3107,
         0.9783, 0.0724],
        [0.0484, 0.7685, 0.5785, 0.8047, 0.2464, 0.3336, 0.4751, 0.7594, 0.5347,
         0.6532, 0.8528, 0.3441, 0.7532, 0.2871, 0.2056, 0.5964, 0.2061, 0.6672,
         0.2814, 0.1722],
        [0.7590, 0.7923, 0.1534, 0.128

C:\Users\hyech\AppData\Local\Temp\ipykernel_19920\3762948969.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matKron = torch.tensor(torch.kron(mat, mat), requires_grad=True)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x20 and 400x400)

In [24]:
x = torch.rand((10, 10))
y = torch.linalg.inv(x)
z = torch.zeros((10, 10))
for i in range(10):
    for j in range(10):
        z[i][j] = y[i][j]
print(x.shape, x.stride(), '\n', x)
print(y.shape, y.stride(), '\n', y)
print(z.shape, z.stride(), '\n', z)

torch.Size([10, 10]) (10, 1) 
 tensor([[9.3497e-01, 4.0005e-01, 2.0105e-02, 2.6490e-01, 9.8470e-01, 4.3822e-01,
         2.0894e-02, 2.5077e-01, 9.6684e-01, 1.2306e-01],
        [7.8026e-01, 9.2803e-01, 4.2527e-01, 1.5272e-01, 7.1347e-01, 1.0540e-01,
         7.2966e-02, 2.8718e-01, 4.1051e-01, 9.5797e-02],
        [9.8772e-01, 1.8273e-01, 1.3484e-01, 8.2902e-01, 8.0842e-04, 4.5023e-01,
         3.2139e-01, 1.3349e-01, 8.9626e-01, 2.3043e-01],
        [8.5540e-02, 2.2020e-01, 1.1609e-01, 1.4752e-01, 4.2841e-01, 2.3119e-02,
         7.2707e-01, 1.2080e-01, 8.3656e-01, 2.2331e-01],
        [9.5180e-01, 6.1940e-01, 9.5660e-01, 2.1658e-01, 9.4941e-01, 3.4822e-01,
         7.8551e-01, 6.2152e-01, 6.1128e-01, 5.7465e-01],
        [6.1736e-01, 8.8028e-01, 2.0126e-01, 4.2004e-01, 5.1344e-01, 6.3661e-01,
         5.6108e-01, 7.3427e-01, 8.9320e-01, 9.4737e-01],
        [5.8891e-01, 4.4999e-01, 4.4679e-01, 5.0741e-01, 6.7626e-01, 3.7722e-01,
         7.1011e-01, 8.5262e-02, 2.1999e-01, 4.5742e-0